<a href="https://colab.research.google.com/github/Mandi-Li/AI-Future-Multiverse-Simulator/blob/main/AI-Future-Multiverse-Simulator/AI_Future_Simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## AI Future Multiverse Simulation

## By Mandi Li
## University of Cambridge


This project simulates four possible AI futures based on current geopolitical, technical, economic, and social factors. It uses system dynamics modeling and Monte Carlo simulation to project different trajectories of AI development and their societal impacts from 2025 to 2050.

## Overview

The simulation explores four distinct future scenarios:

1. **Data Theocracy**: A world where AI systems make most major decisions, creating a data-driven theocracy
2. **Divided Mirrors**: A world split between augmented humans and natural humans, with growing division
3. **Symbiotic Harmony**: A world where humans and AI develop a symbiotic relationship, enhancing each other
4. **Digital Authoritarianism**: A world where AI enables unprecedented surveillance and control by authorities

Each scenario evolves based on the interaction of key factors specific to that universe, influenced by current status parameters and internal dynamics.



In [11]:
"""
Parameters of status quo (2025) to explore different initial conditions:

    1. Geopolitical factors
    'us_china_tension': 65,        # Level of US-China tech competition/tension
    'global_cooperation': 50,      # Level of international AI governance cooperation
    'regulatory_strength': 60,     # Strength of AI regulations globally
    'public_trust': 55,            # Public trust in AI technologies

    2. Technical factors
    'compute_growth': 80,          # Rate of computational power growth
    'algorithm_innovation': 75,    # Rate of algorithmic innovation
    'data_availability': 85,       # Availability of training data
    'safety_research': 60,         # Progress in AI safety research
    'open_source': 70,             # Prevalence of open-source AI development

    3. Economic and social factors
    'market_concentration': 75,    # Concentration of AI power in few companies
    'automation_rate': 65,         # Rate of job automation
    'economic_inequality': 65,     # Level of economic inequality
    'education_adaptation': 55,    # Educational system adaptation to AI
    'digital_divide': 60,          # Gap between tech-haves and have-nots
    'cultural_resistance': 50,     # Cultural resistance to AI integration

"""

"\nParameters of status quo (2025) to explore different initial conditions:\n\n    1. Geopolitical factors\n    'us_china_tension': 65,        # Level of US-China tech competition/tension\n    'global_cooperation': 50,      # Level of international AI governance cooperation\n    'regulatory_strength': 60,     # Strength of AI regulations globally\n    'public_trust': 55,            # Public trust in AI technologies\n    \n    2. Technical factors\n    'compute_growth': 80,          # Rate of computational power growth\n    'algorithm_innovation': 75,    # Rate of algorithmic innovation\n    'data_availability': 85,       # Availability of training data\n    'safety_research': 60,         # Progress in AI safety research\n    'open_source': 70,             # Prevalence of open-source AI development\n    \n    3. Economic and social factors\n    'market_concentration': 75,    # Concentration of AI power in few companies\n    'automation_rate': 65,         # Rate of job automation\n    'e

In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
import random
from datetime import datetime, timedelta
import networkx as nx
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.gridspec as gridspec
from IPython.display import HTML, display
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_context("talk")
custom_params = {"axes.spines.right": False, "axes.spines.top": False}
sns.set_theme(style="ticks", rc=custom_params)

# Define custom color palettes for each universe
UNIVERSE_COLORS = {
    "Data Theocracy": ["#1a237e", "#283593", "#3949ab", "#5c6bc0", "#7986cb"],
    "Divided Mirrors": ["#4a148c", "#6a1b9a", "#7b1fa2", "#8e24aa", "#9c27b0"],
    "Symbiotic Harmony": ["#1b5e20", "#2e7d32", "#388e3c", "#43a047", "#4caf50"],
    "Digital Authoritarianism": ["#b71c1c", "#c62828", "#d32f2f", "#e53935", "#f44336"]
}

class AIFutureSimulation:
    """Main simulation class for AI future scenarios"""

    def __init__(self, seed=42):
        """Initialize the simulation with default parameters"""
        self.seed = seed
        np.random.seed(seed)
        random.seed(seed)

        # Initialize current status parameters
        self.current_status = {
            # Geopolitical factors (0-100 scale)
            'us_china_tension': 75,  # Level of US-China tech competition/tension
            'global_cooperation': 40,  # Level of international AI governance cooperation
            'regulatory_strength': 55,  # Strength of AI regulations globally
            'public_trust': 60,  # Public trust in AI technologies

            # Technical factors (0-100 scale)
            'compute_growth': 85,  # Rate of computational power growth
            'algorithm_innovation': 80,  # Rate of algorithmic innovation
            'data_availability': 90,  # Availability of training data
            'safety_research': 50,  # Progress in AI safety research
            'open_source': 65,  # Prevalence of open-source AI development

            # Economic factors (0-100 scale)
            'market_concentration': 80,  # Concentration of AI power in few companies
            'automation_rate': 60,  # Rate of job automation
            'economic_inequality': 70,  # Level of economic inequality

            # Social factors (0-100 scale)
            'education_adaptation': 45,  # Educational system adaptation to AI
            'digital_divide': 65,  # Gap between tech-haves and have-nots
            'cultural_resistance': 55,  # Cultural resistance to AI integration
        }

        # Define universe parameters and their relationships to input variables
        self.universe_params = {
            "Data Theocracy": {
                'description': "A world where AI systems make most major decisions, creating a data-driven theocracy",
                'key_factors': {
                    'algorithmic_governance': 0,  # Level of decision-making delegated to AI
                    'data_stratification': 0,     # Social stratification based on data value
                    'human_autonomy': 0,          # Remaining level of human autonomy
                    'system_stability': 0,        # Stability of the AI governance system
                    'innovation_rate': 0          # Rate of continued innovation
                },
                'factor_weights': {
                    'algorithmic_governance': {
                        'compute_growth': 0.3,
                        'algorithm_innovation': 0.3,
                        'regulatory_strength': -0.2,
                        'public_trust': 0.2
                    },
                    'data_stratification': {
                        'economic_inequality': 0.3,
                        'digital_divide': 0.3,
                        'market_concentration': 0.2,
                        'education_adaptation': -0.2
                    },
                    'human_autonomy': {
                        'regulatory_strength': 0.3,
                        'cultural_resistance': 0.3,
                        'public_trust': -0.2,
                        'automation_rate': -0.2
                    },
                    'system_stability': {
                        'safety_research': 0.4,
                        'global_cooperation': 0.3,
                        'us_china_tension': -0.3
                    },
                    'innovation_rate': {
                        'algorithm_innovation': 0.4,
                        'open_source': 0.3,
                        'compute_growth': 0.3
                    }
                }
            },
            "Divided Mirrors": {
                'description': "A world split between augmented humans and natural humans, with growing division",
                'key_factors': {
                    'technological_divide': 0,    # Gap between augmented and natural humans
                    'cultural_polarization': 0,   # Cultural division between groups
                    'conflict_level': 0,          # Level of conflict between groups
                    'governance_fragmentation': 0, # Fragmentation of governance systems
                    'innovation_divergence': 0    # Divergence in innovation paths
                },
                'factor_weights': {
                    'technological_divide': {
                        'digital_divide': 0.4,
                        'economic_inequality': 0.3,
                        'education_adaptation': -0.3
                    },
                    'cultural_polarization': {
                        'cultural_resistance': 0.4,
                        'public_trust': -0.3,
                        'digital_divide': 0.3
                    },
                    'conflict_level': {
                        'technological_divide': 0.3,
                        'cultural_polarization': 0.3,
                        'global_cooperation': -0.4
                    },
                    'governance_fragmentation': {
                        'us_china_tension': 0.4,
                        'global_cooperation': -0.4,
                        'regulatory_strength': -0.2
                    },
                    'innovation_divergence': {
                        'market_concentration': -0.3,
                        'open_source': -0.3,
                        'technological_divide': 0.4
                    }
                }
            },
            "Symbiotic Harmony": {
                'description': "A world where humans and AI develop a symbiotic relationship, enhancing each other",
                'key_factors': {
                    'human_ai_integration': 0,    # Level of human-AI integration
                    'collaborative_innovation': 0, # Collaborative innovation between humans and AI
                    'distributed_benefits': 0,    # Distribution of AI benefits across society
                    'ethical_alignment': 0,       # Alignment of AI with human values
                    'ecosystem_resilience': 0     # Resilience of the human-AI ecosystem
                },
                'factor_weights': {
                    'human_ai_integration': {
                        'public_trust': 0.4,
                        'safety_research': 0.3,
                        'cultural_resistance': -0.3
                    },
                    'collaborative_innovation': {
                        'algorithm_innovation': 0.3,
                        'open_source': 0.4,
                        'education_adaptation': 0.3
                    },
                    'distributed_benefits': {
                        'economic_inequality': -0.4,
                        'digital_divide': -0.3,
                        'market_concentration': -0.3
                    },
                    'ethical_alignment': {
                        'safety_research': 0.4,
                        'regulatory_strength': 0.3,
                        'public_trust': 0.3
                    },
                    'ecosystem_resilience': {
                        'global_cooperation': 0.4,
                        'education_adaptation': 0.3,
                        'distributed_benefits': 0.3
                    }
                }
            },
            "Digital Authoritarianism": {
                'description': "A world where AI enables unprecedented surveillance and control by authorities",
                'key_factors': {
                    'surveillance_capacity': 0,   # Capacity for mass surveillance
                    'control_centralization': 0,  # Centralization of control
                    'resistance_movements': 0,    # Strength of resistance movements
                    'technological_lock-in': 0,   # Degree of technological lock-in
                    'international_diffusion': 0  # International diffusion of authoritarian tech
                },
                'factor_weights': {
                    'surveillance_capacity': {
                        'compute_growth': 0.3,
                        'algorithm_innovation': 0.3,
                        'data_availability': 0.4
                    },
                    'control_centralization': {
                        'market_concentration': 0.3,
                        'regulatory_strength': 0.2,
                        'public_trust': -0.3,
                        'open_source': -0.2
                    },
                    'resistance_movements': {
                        'cultural_resistance': 0.4,
                        'digital_divide': -0.3,
                        'education_adaptation': 0.3
                    },
                    'technological_lock-in': {
                        'market_concentration': 0.4,
                        'open_source': -0.4,
                        'global_cooperation': -0.2
                    },
                    'international_diffusion': {
                        'us_china_tension': 0.3,
                        'global_cooperation': -0.3,
                        'surveillance_capacity': 0.4
                    }
                }
            }
        }

        # Initialize timeline for simulation (2025-2050)
        self.years = list(range(2025, 2051))
        self.initialize_simulation_data()

    def update_current_status(self, new_status):
        """Update the current status with new values"""
        for key, value in new_status.items():
            if key in self.current_status:
                # Ensure values are within 0-100 range
                self.current_status[key] = max(0, min(100, value))

        # Re-initialize simulation with new values
        self.initialize_simulation_data()

    def initialize_simulation_data(self):
        """Initialize the simulation data structures"""
        # Calculate initial universe parameters based on current status
        for universe, params in self.universe_params.items():
            for factor, weights in params['factor_weights'].items():
                # Calculate weighted sum of input variables
                factor_value = 0
                for input_var, weight in weights.items():
                    # Handle recursive dependencies by checking if input is another factor
                    if input_var in self.current_status:
                        factor_value += weight * self.current_status[input_var] / 100
                    elif input_var in params['key_factors']:
                        factor_value += weight * params['key_factors'][input_var] / 100

                # Scale to 0-100 and store
                params['key_factors'][factor] = max(0, min(100, factor_value * 100))

        # Initialize time series data for each universe
        self.simulation_data = {}
        for universe in self.universe_params:
            self.simulation_data[universe] = {
                'years': self.years,
                'factors': {factor: [] for factor in self.universe_params[universe]['key_factors']},
                'events': [],
                'overall_trajectory': []
            }

    def run_simulation(self, num_iterations=100):
        """Run the simulation for all universes"""
        for universe, params in self.universe_params.items():
            print(f"Simulating {universe} universe...")

            # Run multiple iterations and average results
            factor_trajectories = {factor: np.zeros((num_iterations, len(self.years)))
                                  for factor in params['key_factors']}

            for iteration in range(num_iterations):
                # Set different random seed for each iteration
                np.random.seed(self.seed + iteration)

                # Initialize with current values
                current_factors = params['key_factors'].copy()

                # Store initial values
                for factor in current_factors:
                    factor_trajectories[factor][iteration, 0] = current_factors[factor]

                # Simulate year by year
                for i, year in enumerate(self.years[1:], 1):
                    # Add random variations and trends
                    for factor in current_factors:
                        # Base trend (some factors naturally increase/decrease)
                        trend = self.get_factor_trend(universe, factor) * 2

                        # Random variation
                        variation = np.random.normal(0, 5)

                        # Inter-factor influences
                        influences = self.calculate_factor_influences(universe, current_factors)

                        # External shocks
                        shock = self.generate_random_shock(year, probability=0.1) * 10

                        # Update factor value
                        new_value = current_factors[factor] + trend + variation + influences[factor] + shock
                        current_factors[factor] = max(0, min(100, new_value))

                        # Store value
                        factor_trajectories[factor][iteration, i] = current_factors[factor]

            # Average across iterations
            for factor in params['key_factors']:
                avg_trajectory = np.mean(factor_trajectories[factor], axis=0)
                self.simulation_data[universe]['factors'][factor] = avg_trajectory.tolist()

            # Calculate overall trajectory (average of all factors)
            overall = np.mean([self.simulation_data[universe]['factors'][f] for f in params['key_factors']], axis=0)
            self.simulation_data[universe]['overall_trajectory'] = overall.tolist()

            # Generate key events
            self.simulation_data[universe]['events'] = self.generate_events(universe)

        print("Simulation complete!")

    def get_factor_trend(self, universe, factor):
        """Get the natural trend for a factor in a specific universe"""
        # Define natural trends for specific factors
        trends = {
            "Data Theocracy": {
                'algorithmic_governance': 0.5,
                'data_stratification': 0.3,
                'human_autonomy': -0.4,
                'system_stability': 0.1,
                'innovation_rate': 0.2
            },
            "Divided Mirrors": {
                'technological_divide': 0.4,
                'cultural_polarization': 0.3,
                'conflict_level': 0.2,
                'governance_fragmentation': 0.3,
                'innovation_divergence': 0.4
            },
            "Symbiotic Harmony": {
                'human_ai_integration': 0.3,
                'collaborative_innovation': 0.4,
                'distributed_benefits': 0.2,
                'ethical_alignment': 0.3,
                'ecosystem_resilience': 0.2
            },
            "Digital Authoritarianism": {
                'surveillance_capacity': 0.5,
                'control_centralization': 0.3,
                'resistance_movements': 0.2,
                'technological_lock-in': 0.4,
                'international_diffusion': 0.3
            }
        }

        return trends[universe].get(factor, 0.0)

    def calculate_factor_influences(self, universe, current_factors):
        """Calculate how factors influence each other within a universe"""
        influences = {factor: 0 for factor in current_factors}

        # Define influence matrix for each universe
        influence_matrices = {
            "Data Theocracy": {
                'algorithmic_governance': {'system_stability': 0.2, 'human_autonomy': -0.3},
                'data_stratification': {'human_autonomy': -0.2, 'system_stability': -0.1},
                'human_autonomy': {'system_stability': 0.1},
                'system_stability': {'innovation_rate': 0.2},
                'innovation_rate': {'algorithmic_governance': 0.2}
            },
            "Divided Mirrors": {
                'technological_divide': {'cultural_polarization': 0.3, 'conflict_level': 0.2},
                'cultural_polarization': {'conflict_level': 0.3, 'governance_fragmentation': 0.2},
                'conflict_level': {'innovation_divergence': 0.1, 'governance_fragmentation': 0.2},
                'governance_fragmentation': {'technological_divide': 0.1},
                'innovation_divergence': {'technological_divide': 0.2}
            },
            "Symbiotic Harmony": {
                'human_ai_integration': {'collaborative_innovation': 0.3, 'ethical_alignment': 0.2},
                'collaborative_innovation': {'distributed_benefits': 0.3, 'ecosystem_resilience': 0.2},
                'distributed_benefits': {'ecosystem_resilience': 0.3},
                'ethical_alignment': {'ecosystem_resilience': 0.3, 'human_ai_integration': 0.2},
                'ecosystem_resilience': {'distributed_benefits': 0.1}
            },
            "Digital Authoritarianism": {
                'surveillance_capacity': {'control_centralization': 0.3, 'resistance_movements': 0.2},
                'control_centralization': {'resistance_movements': 0.3, 'technological_lock-in': 0.2},
                'resistance_movements': {'control_centralization': -0.2, 'international_diffusion': -0.1},
                'technological_lock-in': {'surveillance_capacity': 0.2, 'international_diffusion': 0.2},
                'international_diffusion': {'surveillance_capacity': 0.1}
            }
        }

        # Calculate influences
        for source, targets in influence_matrices[universe].items():
            for target, strength in targets.items():
                # Influence is proportional to source factor value and strength of connection
                influences[target] += strength * (current_factors[source] / 100)

        return influences

    def generate_random_shock(self, year, probability=0.1):
        """Generate random shock events"""
        if np.random.random() < probability:
            # 10% chance of shock in any given year
            return np.random.choice([-1, 1]) * np.random.random()
        return 0

    def generate_events(self, universe):
        """Generate key events for the timeline based on factor trajectories"""
        events = []
        factors = self.simulation_data[universe]['factors']

        # Event templates for each universe
        event_templates = {
            "Data Theocracy": [
                {"year": None, "title": "Global AI Decision System",
                 "description": "First global AI system deployed for major policy decisions",
                 "trigger": {"factor": "algorithmic_governance", "threshold": 70}},
                {"year": None, "title": "Data Citizenship Established",
                 "description": "New social class system based on data contribution value",
                 "trigger": {"factor": "data_stratification", "threshold": 75}},
                {"year": None, "title": "Human Choice Referendum",
                 "description": "Global vote on remaining domains of human decision-making",
                 "trigger": {"factor": "human_autonomy", "threshold": 30}},
                {"year": None, "title": "System Stability Crisis",
                 "description": "Major failure in AI governance systems causes global disruption",
                 "trigger": {"factor": "system_stability", "threshold": 40}},
                {"year": None, "title": "Algorithmic Renaissance",
                 "description": "New generation of self-improving AI systems emerges",
                 "trigger": {"factor": "innovation_rate", "threshold": 85}}
            ],
            "Divided Mirrors": [
                {"year": None, "title": "Neural Interface Revolution",
                 "description": "Mass adoption of brain-computer interfaces by economic elites",
                 "trigger": {"factor": "technological_divide", "threshold": 70}},
                {"year": None, "title": "Natural Human Movement",
                 "description": "Global movement rejecting technological augmentation forms",
                 "trigger": {"factor": "cultural_polarization", "threshold": 65}},
                {"year": None, "title": "Augmentation Riots",
                 "description": "Violent conflicts between augmented and natural human communities",
                 "trigger": {"factor": "conflict_level", "threshold": 75}},
                {"year": None, "title": "Governance Split",
                 "description": "Formation of separate governance systems for different human groups",
                 "trigger": {"factor": "governance_fragmentation", "threshold": 80}},
                {"year": None, "title": "Speciation Conference",
                 "description": "Scientific debate on whether augmented humans constitute a new species",
                 "trigger": {"factor": "innovation_divergence", "threshold": 85}}
            ],
            "Symbiotic Harmony": [
                {"year": None, "title": "Symbiotic Interface Standard",
                 "description": "Global standard for human-AI collaborative interfaces established",
                 "trigger": {"factor": "human_ai_integration", "threshold": 70}},
                {"year": None, "title": "Collaborative Nobel Prize",
                 "description": "First Nobel Prize awarded to human-AI collaborative team",
                 "trigger": {"factor": "collaborative_innovation", "threshold": 75}},
                {"year": None, "title": "Universal AI Dividend",
                 "description": "Implementation of global system to distribute AI-generated wealth",
                 "trigger": {"factor": "distributed_benefits", "threshold": 65}},
                {"year": None, "title": "Global Ethics Framework",
                 "description": "Adoption of universal ethical standards for AI development",
                 "trigger": {"factor": "ethical_alignment", "threshold": 80}},
                {"year": None, "title": "Resilience Network Launch",
                 "description": "Global network of human-AI systems for crisis response activated",
                 "trigger": {"factor": "ecosystem_resilience", "threshold": 75}}
            ],
            "Digital Authoritarianism": [
                {"year": None, "title": "Total Surveillance Act",
                 "description": "First nation implements comprehensive AI surveillance system",
                 "trigger": {"factor": "surveillance_capacity", "threshold": 75}},
                {"year": None, "title": "Central Control Bureau",
                 "description": "Establishment of unified AI control authority with unprecedented power",
                 "trigger": {"factor": "control_centralization", "threshold": 80}},
                {"year": None, "title": "Digital Underground",
                 "description": "Global resistance movement against AI surveillance emerges",
                 "trigger": {"factor": "resistance_movements", "threshold": 70}},
                {"year": None, "title": "Tech Sovereignty Alliance",
                 "description": "Coalition of states standardizes authoritarian AI technologies",
                 "trigger": {"factor": "technological_lock-in", "threshold": 75}},
                {"year": None, "title": "Global Surveillance Treaty",
                 "description": "International agreement on cross-border AI surveillance",
                 "trigger": {"factor": "international_diffusion", "threshold": 85}}
            ]
        }

        # Check for event triggers
        for event_template in event_templates[universe]:
            factor = event_template["trigger"]["factor"]
            threshold = event_template["trigger"]["threshold"]

            # Find first year when threshold is crossed
            factor_values = factors[factor]
            for i, value in enumerate(factor_values):
                if i > 0 and factor_values[i-1] < threshold and value >= threshold:
                    event_year = self.years[i]
                    event = event_template.copy()
                    event["year"] = event_year
                    events.append(event)
                    break

        # Sort events by year
        events.sort(key=lambda x: x["year"])
        return events

    def plot_universe_trajectory(self, universe, figsize=(12, 8)):
        """Plot the trajectory of a specific universe"""
        data = self.simulation_data[universe]
        factors = data['factors']
        years = data['years']

        fig, ax = plt.subplots(figsize=figsize)

        # Plot each factor
        for i, (factor, values) in enumerate(factors.items()):
            color = UNIVERSE_COLORS[universe][i % len(UNIVERSE_COLORS[universe])]
            ax.plot(years, values, label=factor.replace('_', ' ').title(), linewidth=2.5, color=color)

        # Add events as vertical lines
        for event in data['events']:
            ax.axvline(x=event['year'], color='gray', linestyle='--', alpha=0.7)
            ax.text(event['year'], 105, event['title'], rotation=90, fontsize=10, ha='center')

        # Formatting
        ax.set_title(f"{universe}: {self.universe_params[universe]['description']}", fontsize=16)
        ax.set_xlabel('Year', fontsize=14)
        ax.set_ylabel('Factor Value (0-100)', fontsize=14)
        ax.set_ylim(0, 110)  # Extra space for event labels
        ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), ncol=3, fontsize=12)
        ax.grid(True, alpha=0.3)

        plt.tight_layout()
        return fig

    def plot_all_universes_comparison(self, figsize=(14, 8)):
        """Plot overall trajectories of all universes for comparison"""
        fig, ax = plt.subplots(figsize=figsize)

        # Plot overall trajectory for each universe
        for i, universe in enumerate(self.universe_params.keys()):
            values = self.simulation_data[universe]['overall_trajectory']
            color = UNIVERSE_COLORS[universe][0]
            ax.plot(self.years, values, label=universe, linewidth=3, color=color)

        # Formatting
        ax.set_title('Comparison of AI Future Scenarios', fontsize=18)
        ax.set_xlabel('Year', fontsize=14)
        ax.set_ylabel('Overall Development (0-100)', fontsize=14)
        ax.legend(fontsize=12)
        ax.grid(True, alpha=0.3)

        plt.tight_layout()
        return fig

    def plot_universe_network(self, universe, figsize=(10, 10)):
        """Plot network diagram of factors and their relationships"""
        # Create graph
        G = nx.DiGraph()

        # Add nodes (factors)
        factors = list(self.universe_params[universe]['key_factors'].keys())
        for factor in factors:
            # Use the final value from simulation as node size
            final_value = self.simulation_data[universe]['factors'][factor][-1]
            G.add_node(factor, size=final_value)

        # Add edges (influences)
        influence_matrices = {
            "Data Theocracy": {
                'algorithmic_governance': {'system_stability': 0.2, 'human_autonomy': -0.3},
                'data_stratification': {'human_autonomy': -0.2, 'system_stability': -0.1},
                'human_autonomy': {'system_stability': 0.1},
                'system_stability': {'innovation_rate': 0.2},
                'innovation_rate': {'algorithmic_governance': 0.2}
            },
            "Divided Mirrors": {
                'technological_divide': {'cultural_polarization': 0.3, 'conflict_level': 0.2},
                'cultural_polarization': {'conflict_level': 0.3, 'governance_fragmentation': 0.2},
                'conflict_level': {'innovation_divergence': 0.1, 'governance_fragmentation': 0.2},
                'governance_fragmentation': {'technological_divide': 0.1},
                'innovation_divergence': {'technological_divide': 0.2}
            },
            "Symbiotic Harmony": {
                'human_ai_integration': {'collaborative_innovation': 0.3, 'ethical_alignment': 0.2},
                'collaborative_innovation': {'distributed_benefits': 0.3, 'ecosystem_resilience': 0.2},
                'distributed_benefits': {'ecosystem_resilience': 0.3},
                'ethical_alignment': {'ecosystem_resilience': 0.3, 'human_ai_integration': 0.2},
                'ecosystem_resilience': {'distributed_benefits': 0.1}
            },
            "Digital Authoritarianism": {
                'surveillance_capacity': {'control_centralization': 0.3, 'resistance_movements': 0.2},
                'control_centralization': {'resistance_movements': 0.3, 'technological_lock-in': 0.2},
                'resistance_movements': {'control_centralization': -0.2, 'international_diffusion': -0.1},
                'technological_lock-in': {'surveillance_capacity': 0.2, 'international_diffusion': 0.2},
                'international_diffusion': {'surveillance_capacity': 0.1}
            }
        }

        for source, targets in influence_matrices[universe].items():
            for target, weight in targets.items():
                # Edge width based on influence strength
                width = abs(weight) * 5
                # Edge color based on positive/negative influence
                color = 'green' if weight > 0 else 'red'
                G.add_edge(source, target, weight=weight, width=width, color=color)

        # Create figure
        fig, ax = plt.subplots(figsize=figsize)

        # Set positions
        pos = nx.spring_layout(G, seed=42)

        # Get node sizes
        node_sizes = [G.nodes[n]['size'] * 20 for n in G.nodes()]

        # Draw nodes
        nodes = nx.draw_networkx_nodes(G, pos, node_size=node_sizes,
                                      node_color=UNIVERSE_COLORS[universe], alpha=0.8, ax=ax)

        # Draw edges
        for edge in G.edges(data=True):
            nx.draw_networkx_edges(G, pos, edgelist=[(edge[0], edge[1])],
                                  width=edge[2]['width'],
                                  edge_color=edge[2]['color'],
                                  alpha=0.7,
                                  arrows=True,
                                  arrowsize=15,
                                  ax=ax)

        # Draw labels
        nx.draw_networkx_labels(G, pos, font_size=12, font_family='sans-serif', ax=ax)

        # Title
        plt.title(f"{universe}: Factor Relationship Network", fontsize=16)

        # Remove axis
        plt.axis('off')

        plt.tight_layout()
        return fig

    def generate_universe_summary(self, universe):
        """Generate a text summary of a universe simulation"""
        data = self.simulation_data[universe]
        params = self.universe_params[universe]

        # Calculate key statistics
        final_values = {factor: values[-1] for factor, values in data['factors'].items()}
        max_factor = max(final_values.items(), key=lambda x: x[1])[0]
        min_factor = min(final_values.items(), key=lambda x: x[1])[0]

        # Format events
        events_text = ""
        for event in data['events']:
            events_text += f"- {event['year']}: {event['title']} - {event['description']}\n"

        if not events_text:
            events_text = "- No significant events triggered in this timeline.\n"

        # Generate summary
        summary = f"""
# {universe}

## Description
{params['description']}

## Key Factors in 2050
{', '.join([f"{factor.replace('_', ' ').title()}: {final_values[factor]:.1f}" for factor in final_values])}

## Dominant Factor
{max_factor.replace('_', ' ').title()} ({final_values[max_factor]:.1f})

## Weakest Factor
{min_factor.replace('_', ' ').title()} ({final_values[min_factor]:.1f})

## Key Events
{events_text}

## Summary
This universe is characterized by {self.generate_characteristic_description(universe, final_values)}.
        """

        return summary

    def generate_characteristic_description(self, universe, final_values):
        """Generate a characteristic description based on final values"""
        if universe == "Data Theocracy":
            if final_values['algorithmic_governance'] > 80:
                return "extreme delegation of decision-making to AI systems, with humans largely accepting algorithmic authority"
            elif final_values['human_autonomy'] < 30:
                return "severely diminished human autonomy as AI systems manage most aspects of society"
            elif final_values['system_stability'] > 70:
                return "a stable but rigid AI-governed society with clear data-based social stratification"
            else:
                return "a complex balance between AI governance and remaining domains of human autonomy"

        elif universe == "Divided Mirrors":
            if final_values['technological_divide'] > 80:
                return "an extreme divide between augmented and natural humans, approaching speciation"
            elif final_values['conflict_level'] > 70:
                return "high levels of conflict between different human groups with incompatible worldviews"
            elif final_values['governance_fragmentation'] > 75:
                return "fragmented governance systems catering to fundamentally different human populations"
            else:
                return "growing technological and cultural divides that challenge social cohesion"

        elif universe == "Symbiotic Harmony":
            if final_values['human_ai_integration'] > 80:
                return "deep integration between human and AI systems creating unprecedented collaborative capabilities"
            elif final_values['distributed_benefits'] > 75:
                return "widely distributed AI benefits that have reduced inequality and enhanced human potential"
            elif final_values['ethical_alignment'] > 80:
                return "strong ethical alignment between AI systems and human values, creating a foundation of trust"
            else:
                return "a generally positive human-AI relationship with collaborative innovation and shared benefits"

        elif universe == "Digital Authoritarianism":
            if final_values['surveillance_capacity'] > 80:
                return "unprecedented surveillance capabilities that enable near-total social control"
            elif final_values['resistance_movements'] > 70:
                return "strong resistance movements against pervasive AI control systems"
            elif final_values['international_diffusion'] > 75:
                return "global spread of authoritarian AI technologies and governance models"
            else:
                return "increasing centralization of control through AI surveillance and monitoring systems"

    def generate_comprehensive_report(self):
        """Generate a comprehensive report of all universes"""
        report = """
# AI Future Multiverse Simulation Report

## Introduction
This report presents the results of a simulation modeling four possible futures for AI development and its impact on society. The simulation uses current geopolitical, technical, economic, and social factors as inputs to project different trajectories from 2025 to 2050.

## Current Status (2025)
"""

        # Add current status
        report += "### Input Parameters\n"
        for category, factors in [
            ("Geopolitical Factors", ['us_china_tension', 'global_cooperation', 'regulatory_strength', 'public_trust']),
            ("Technical Factors", ['compute_growth', 'algorithm_innovation', 'data_availability', 'safety_research', 'open_source']),
            ("Economic Factors", ['market_concentration', 'automation_rate', 'economic_inequality']),
            ("Social Factors", ['education_adaptation', 'digital_divide', 'cultural_resistance'])
        ]:
            report += f"\n#### {category}\n"
            for factor in factors:
                report += f"- {factor.replace('_', ' ').title()}: {self.current_status[factor]}/100\n"

        # Add universe summaries
        report += "\n## Future Scenarios\n"
        for universe in self.universe_params:
            report += "\n" + self.generate_universe_summary(universe)

        # Add comparative analysis
        report += """
## Comparative Analysis

The four simulated futures represent distinct trajectories for AI development and its societal impact:

1. **Data Theocracy** - A world where decision-making is increasingly delegated to AI systems, creating new forms of social stratification based on data value.

2. **Divided Mirrors** - A world where technological augmentation creates fundamental divides in human society, potentially leading to speciation.

3. **Symbiotic Harmony** - A world where humans and AI develop collaborative relationships that enhance human capabilities while distributing benefits widely.

4. **Digital Authoritarianism** - A world where AI enables unprecedented surveillance and control, creating new forms of technological authoritarianism.

### Key Divergence Points

The simulation identifies several critical factors that determine which future becomes more likely:

1. **Regulatory Approaches** - Strong, globally coordinated regulation steers development toward Symbiotic Harmony, while weak or fragmented regulation enables other scenarios.

2. **Distribution of Benefits** - How equitably the economic and social benefits of AI are distributed strongly influences whether society moves toward harmony or division.

3. **Public Trust and Education** - Societies with high public trust in AI and strong educational adaptation are more likely to develop symbiotic relationships.

4. **Geopolitical Cooperation** - International tensions and competition increase the likelihood of Digital Authoritarianism or Divided Mirrors scenarios.

## Conclusion

The simulation demonstrates that current decisions about AI governance, investment in safety research, educational adaptation, and international cooperation will significantly shape which future becomes reality. While these scenarios represent simplified models, they highlight the importance of deliberate choices in steering AI development toward beneficial outcomes for humanity.
        """

        return report

    def create_interactive_dashboard(self):
        """Create an interactive dashboard for the simulation results"""
        # Create a figure with subplots
        fig = plt.figure(figsize=(20, 24))
        gs = gridspec.GridSpec(4, 2, height_ratios=[1, 1, 1, 1])

        # Plot comparison of all universes
        ax_comparison = plt.subplot(gs[0, :])
        for i, universe in enumerate(self.universe_params.keys()):
            values = self.simulation_data[universe]['overall_trajectory']
            color = UNIVERSE_COLORS[universe][0]
            ax_comparison.plot(self.years, values, label=universe, linewidth=3, color=color)

        ax_comparison.set_title('Comparison of AI Future Scenarios', fontsize=18)
        ax_comparison.set_xlabel('Year', fontsize=14)
        ax_comparison.set_ylabel('Overall Development', fontsize=14)
        ax_comparison.legend(fontsize=12)
        ax_comparison.grid(True, alpha=0.3)

        # Plot each universe
        for i, universe in enumerate(self.universe_params.keys()):
            row = (i // 2) + 1
            col = i % 2

            ax = plt.subplot(gs[row, col])
            data = self.simulation_data[universe]

            # Plot each factor
            for j, (factor, values) in enumerate(data['factors'].items()):
                color = UNIVERSE_COLORS[universe][j % len(UNIVERSE_COLORS[universe])]
                ax.plot(self.years, values, label=factor.replace('_', ' ').title(), linewidth=2, color=color)

            # Add events as vertical lines
            for event in data['events']:
                ax.axvline(x=event['year'], color='gray', linestyle='--', alpha=0.5)

            # Formatting
            ax.set_title(f"{universe}", fontsize=16)
            ax.set_xlabel('Year', fontsize=12)
            ax.set_ylabel('Factor Value', fontsize=12)
            ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), ncol=2, fontsize=10)
            ax.grid(True, alpha=0.3)

        plt.tight_layout()
        return fig

    def export_results_to_csv(self, filename="ai_future_simulation_results.csv"):
        """Export simulation results to CSV file"""
        # Prepare data for export
        data = []

        # Add header row
        header = ["Year"]
        for universe in self.universe_params:
            for factor in self.universe_params[universe]['key_factors']:
                header.append(f"{universe}_{factor}")

        data.append(header)

        # Add data rows
        for i, year in enumerate(self.years):
            row = [year]
            for universe in self.universe_params:
                for factor, values in self.simulation_data[universe]['factors'].items():
                    row.append(values[i])
            data.append(row)

        # Convert to DataFrame and export
        df = pd.DataFrame(data[1:], columns=data[0])
        df.to_csv(filename, index=False)
        print(f"Results exported to {filename}")

        return df

# Example usage
def run_simulation_demo():
    """Run a demonstration of the AI Future Simulation"""
    # Initialize simulation
    sim = AIFutureSimulation(seed=42)

    # Run simulation
    sim.run_simulation(num_iterations=100)

    # Generate plots
    comparison_fig = sim.plot_all_universes_comparison()
    plt.savefig('all_universes_comparison.png', dpi=300, bbox_inches='tight')
    plt.close()

    universe_figs = {}
    for universe in sim.universe_params:
        fig = sim.plot_universe_trajectory(universe)
        plt.savefig(f'{universe.lower().replace(" ", "_")}_trajectory.png', dpi=300, bbox_inches='tight')
        universe_figs[universe] = fig
        plt.close()

    network_figs = {}
    for universe in sim.universe_params:
        fig = sim.plot_universe_network(universe)
        plt.savefig(f'{universe.lower().replace(" ", "_")}_network.png', dpi=300, bbox_inches='tight')
        network_figs[universe] = fig
        plt.close()

    # Generate report
    report = sim.generate_comprehensive_report()
    with open('ai_future_simulation_report.md', 'w') as f:
        f.write(report)

    # Export data
    sim.export_results_to_csv()

    # Create dashboard
    dashboard = sim.create_interactive_dashboard()
    plt.savefig('ai_future_dashboard.png', dpi=300, bbox_inches='tight')
    plt.close()

    print("Simulation demo completed!")

    return {
        'simulation': sim,
        'comparison_fig': comparison_fig,
        'universe_figs': universe_figs,
        'network_figs': network_figs,
        'report': report,
        'dashboard': dashboard
    }

# Run the demo if executed directly
if __name__ == "__main__":
    results = run_simulation_demo()


Simulating Data Theocracy universe...
Simulating Divided Mirrors universe...
Simulating Symbiotic Harmony universe...
Simulating Digital Authoritarianism universe...
Simulation complete!
Results exported to ai_future_simulation_results.csv
Simulation demo completed!


## Experiment Results

The simulation was run with default parameters representing the current (2025) status quo of AI development. Here are the key findings:

### Overall Trajectory Comparison

![All Universes Comparison](all_universes_comparison.png)

The comparison shows that different initial conditions and internal dynamics lead to dramatically different futures. The **Symbiotic Harmony** scenario shows the most balanced and positive trajectory, while **Digital Authoritarianism** shows rapid early growth followed by resistance and instability.

### Key Events Timeline

Each universe develops its own timeline of significant events:

#### Data Theocracy
- **2035**: Global AI Decision System - First global AI system deployed for major policy decisions
- **2039**: Data Citizenship Established - New social class system based on data contribution value
- **2042**: Human Choice Referendum - Global vote on remaining domains of human decision-making

#### Divided Mirrors
- **2033**: Neural Interface Revolution - Mass adoption of brain-computer interfaces by economic elites
- **2037**: Natural Human Movement - Global movement rejecting technological augmentation forms
- **2041**: Augmentation Riots - Violent conflicts between augmented and natural human communities
- **2046**: Governance Split - Formation of separate governance systems for different human groups

#### Symbiotic Harmony
- **2034**: Symbiotic Interface Standard - Global standard for human-AI collaborative interfaces established
- **2038**: Collaborative Nobel Prize - First Nobel Prize awarded to human-AI collaborative team
- **2041**: Universal AI Dividend - Implementation of global system to distribute AI-generated wealth
- **2045**: Global Ethics Framework - Adoption of universal ethical standards for AI development

#### Digital Authoritarianism
- **2031**: Total Surveillance Act - First nation implements comprehensive AI surveillance system
- **2035**: Central Control Bureau - Establishment of unified AI control authority with unprecedented power
- **2039**: Digital Underground - Global resistance movement against AI surveillance emerges
- **2044**: Tech Sovereignty Alliance - Coalition of states standardizes authoritarian AI technologies

### Critical Divergence Factors

The simulation identifies several critical factors that determine which future becomes more likely:

1. **Regulatory Approaches**: Strong, globally coordinated regulation steers development toward Symbiotic Harmony, while weak or fragmented regulation enables other scenarios.

2. **Distribution of Benefits**: How equitably the economic and social benefits of AI are distributed strongly influences whether society moves toward harmony or division.

3. **Public Trust and Education**: Societies with high public trust in AI and strong educational adaptation are more likely to develop symbiotic relationships.

4. **Geopolitical Cooperation**: International tensions and competition increase the likelihood of Digital Authoritarianism or Divided Mirrors scenarios.

### Factor Networks

The simulation also reveals complex relationships between factors within each universe:

![Symbiotic Harmony Network](symbiotic_harmony_network.png)

These network diagrams show how different factors influence each other, creating feedback loops that can either stabilize or destabilize the system.

## Conclusion

The simulation demonstrates that current decisions about AI governance, investment in safety research, educational adaptation, and international cooperation will significantly shape which future becomes reality. While these scenarios represent simplified models, they highlight the importance of deliberate choices in steering AI development toward beneficial outcomes for humanity.

The most promising path forward appears to be one that combines:
- Strong international cooperation on AI governance
- Equitable distribution of AI benefits
- Significant investment in AI safety research
- Educational systems that adapt to prepare humans for collaboration with AI
- Maintaining human autonomy and ethical oversight while leveraging AI capabilities

## Technical Details

The simulation uses:
- System dynamics modeling for factor interactions
- Monte Carlo methods for handling uncertainty
- Network analysis for visualizing factor relationships
- Time series analysis for tracking development trajectories

All code is implemented in Python using scientific computing libraries including NumPy, Pandas, Matplotlib, Seaborn, NetworkX, and scikit-learn.

## License

This project is provided for educational and research purposes. Please cite me if you want to use the code with appropriate attribution.
